# Detecção de Bot do Twitter

Giovanna Furlan Torres

Turma 4 - Sistemas de Informação

## Problemática

Treinar uma rede neural BERT no Keras para detectar bots no Twitter utilizando o dataset Twitter-Bot Detection, disponível no Kaggle.

https://www.kaggle.com/datasets/goyaladi/twitter-bot-detection-dataset

# Setup

A configuração de setup é o processo de preparar e organizar o ambiente para uso.

## Bibliotecas

Essa etapa envolve a instalação de bibliotecas e configuração de outros ajustes necessários.

In [1]:
# Instale o unidecode
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 13.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import gdown
import plotly.express as px
import string
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
import plotly.graph_objects as go
from unidecode import unidecode
import re
from transformers import Trainer, TrainingArguments
import torch
from sklearn.metrics import confusion_matrix, classification_report
import plotly.graph_objs as go
import plotly.figure_factory as ff

## Importação dos dados

Para garantir o acesso aos dados por todos, sem a necessidade de permissões, foi utilizado o gdown para consumir esse dataset.

In [3]:
arquivo_destino_colab = "dataset.csv"

doc_id = "1PP8_BRxU5XSVYcM2fo3l0WmOl6O2OvCn"

URL = f"https://drive.google.com/uc?id={doc_id}"

gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1PP8_BRxU5XSVYcM2fo3l0WmOl6O2OvCn
To: /content/dataset.csv
100%|██████████| 7.46M/7.46M [00:00<00:00, 107MB/s]


'dataset.csv'

In [4]:
dados = pd.read_csv(arquivo_destino_colab)
dados

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention
...,...,...,...,...,...,...,...,...,...,...,...
49995,491196,uberg,Want but put card direction know miss former h...,64,0,9911,True,1,Lake Kimberlyburgh,2023-04-20 11:06:26,teach quality ten education any
49996,739297,jessicamunoz,Provide whole maybe agree church respond most ...,18,5,9900,False,1,Greenbury,2022-10-18 03:57:35,add walk among believe
49997,674475,lynncunningham,Bring different everyone international capital...,43,3,6313,True,1,Deborahfort,2020-07-08 03:54:08,onto admit artist first
49998,167081,richardthompson,Than about single generation itself seek sell ...,45,1,6343,False,0,Stephenside,2022-03-22 12:13:44,star


## Informações Gerais

In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   User ID         50000 non-null  int64 
 1   Username        50000 non-null  object
 2   Tweet           50000 non-null  object
 3   Retweet Count   50000 non-null  int64 
 4   Mention Count   50000 non-null  int64 
 5   Follower Count  50000 non-null  int64 
 6   Verified        50000 non-null  bool  
 7   Bot Label       50000 non-null  int64 
 8   Location        50000 non-null  object
 9   Created At      50000 non-null  object
 10  Hashtags        41659 non-null  object
dtypes: bool(1), int64(5), object(5)
memory usage: 3.9+ MB


### **Sumário Estatístico**

Abaixo, apresenta-se uma visão geral dos dados olímpicos:

In [6]:
num_amostras, num_features = dados.shape

- **Número de amostras:** Indica a quantidade total de registros no conjunto de dados.

In [7]:
print(f"Número de amostras: {num_amostras}")

Número de amostras: 50000


- **Número de features:** Mostra o total de variáveis presentes em cada amostra.


In [8]:
print(f"Número de features: {num_features}")

Número de features: 11


- **Tipos de dados das features:** Exibe os tipos de dados associados a cada coluna do DataFrame.

In [9]:
print("\nTipos de dados das features:")
print(dados.dtypes)


Tipos de dados das features:
User ID            int64
Username          object
Tweet             object
Retweet Count      int64
Mention Count      int64
Follower Count     int64
Verified            bool
Bot Label          int64
Location          object
Created At        object
Hashtags          object
dtype: object


- **Descrição estatística das features numéricas:** Fornece estatísticas descritivas como média, desvio padrão, valores mínimo e máximo, entre outros, para as variáveis numéricas.

In [10]:
print("\nDescrição estatística das features numéricas:")
print(dados.describe())


Descrição estatística das features numéricas:
             User ID  Retweet Count  Mention Count  Follower Count  \
count   50000.000000    50000.00000   50000.000000    50000.000000   
mean   548890.680540       50.00560       2.513760     4988.602380   
std    259756.681425       29.18116       1.708563     2878.742898   
min    100025.000000        0.00000       0.000000        0.000000   
25%    323524.250000       25.00000       1.000000     2487.750000   
50%    548147.000000       50.00000       3.000000     4991.500000   
75%    772983.000000       75.00000       4.000000     7471.000000   
max    999995.000000      100.00000       5.000000    10000.000000   

          Bot Label  
count  50000.000000  
mean       0.500360  
std        0.500005  
min        0.000000  
25%        0.000000  
50%        1.000000  
75%        1.000000  
max        1.000000  


- **Valores ausentes por coluna:** Lista a quantidade de valores ausentes em cada coluna do DataFrame.

In [11]:
print("\nValores ausentes por coluna:")
print(dados.isnull().sum())


Valores ausentes por coluna:
User ID              0
Username             0
Tweet                0
Retweet Count        0
Mention Count        0
Follower Count       0
Verified             0
Bot Label            0
Location             0
Created At           0
Hashtags          8341
dtype: int64


- **Valores únicos em cada coluna:** Apresenta o número de valores distintos encontrados em cada coluna.

In [12]:
print("\nValores únicos em cada coluna:")
print(dados.nunique())


Valores únicos em cada coluna:
User ID           50000
Username          40566
Tweet             50000
Retweet Count       101
Mention Count         6
Follower Count     9923
Verified              2
Bot Label             2
Location          25199
Created At        49989
Hashtags          34247
dtype: int64


# Data engineering (EXTRA)


O código abaixo padroniza os nomes das colunas, preenche valores nulos em `hashtags` com `'None'` e converte todos os valores da coluna `location` para minúsculas.

In [13]:
# Alterando cada título para uma versão melhor (minúsculas + "_" em vez de espaços)
dados.columns = [col.lower().replace(" ", "_") for col in dados.columns]

# Preenchendo dados NaN
dados['hashtags'] = dados['hashtags'].fillna('None')

# Removendo maiúsculas
dados['location'] = dados['location'].str.lower()

Abaixo converte-se a coluna `created_at` para o formato datetime e, em seguida, cria novas colunas para armazenar o ano, mês, dia e hora extraídos dessa data.

In [14]:
# Converte a coluna 'created_at' para datetime
dados['created_at'] = pd.to_datetime(dados['created_at'], errors='coerce')

# Extrai componentes de tempo diretamente usando o atributo .dt
dados['created_at_year'] = dados['created_at'].dt.year
dados['created_at_month'] = dados['created_at'].dt.month
dados['created_at_day'] = dados['created_at'].dt.day
dados['created_at_hour'] = dados['created_at'].dt.hour

Definição de uma função para remover caracteres especiais (manter letras, números e espaços) dos tweets e aplicar essa função à coluna `tweet`.

In [15]:
# Função para remover pontuação e acentos
def clean_tweet(tweet):
    # Remove acentos
    tweet = unidecode(tweet)
    # Remove pontuação
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    return tweet

# Função para remover caracteres especiais
def clean_tweet(tweet):
    # Remove caracteres especiais (mantém letras, números e espaços)
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet)
    return tweet

# Aplica a função à coluna 'tweet'
dados['tweet'] = dados['tweet'].apply(clean_tweet)

Exibição do dataframe após as modificações nos dados

In [16]:
dados.head()

,user_id,username,tweet,retweet_count,mention_count,follower_count,verified,bot_label,location,created_at,hashtags,created_at_year,created_at_month,created_at_day,created_at_hour
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,adkinston,2020-05-11 15:29:50,None,2020,5,11,15
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,sanderston,2022-11-26 05:18:10,both live,2022,11,26,5
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,harrisonfurt,2022-08-08 03:16:54,phone ahead,2022,8,8,3
3,696168,pmason,Just cover eight opportunity strong policy which,54,5,2242,True,1,martinezberg,2021-08-14 22:27:05,ever quickly new I,2021,8,14,22
4,704441,noah87,Animal sign six data good or,26,3,8438,False,1,camachoville,2020-04-13 21:24:21,foreign mention,2020,4,13,21


# Matriz de Correlação

Mostra o grau de relação (correlação) entre diferentes variáveis dos dados, indicando como uma variável se comporta em relação a outra. Os valores variam de -1 a 1, onde 1 indica uma correlação positiva perfeita, -1 uma correlação negativa perfeita, e 0 indica ausência de correlação.

In [17]:
# Calcular a matriz de correlação apenas para as colunas numéricas
correlation_matrix = dados[['retweet_count', 'mention_count', 'follower_count', 'bot_label']].corr()

# Converter a matriz de correlação para um formato adequado para plotagem
correlation_matrix = correlation_matrix.reset_index().melt(id_vars='index')
correlation_matrix.columns = ['Feature1', 'Feature2', 'Correlation']

# Criar a tabela pivot
correlation_pivot = correlation_matrix.pivot_table(index='Feature1', columns='Feature2', values='Correlation')

# Criar o gráfico de calor da matriz de correlação
fig = px.imshow(
    correlation_pivot,
    color_continuous_scale='Viridis',
    title='Matriz de Correlação',
    labels={'x': 'Features', 'y': 'Features'},
    text_auto=True  # Adiciona os valores de correlação ao gráfico
)

# Exibir o gráfico
fig.show()


Na matriz apresenta-se cinco variáveis: `bot_label`, `follower_count`, `mention_count`, `retweet_count` e `features`. Sendo possível observar:

1. Todos os valores na diagonal principal são 1, pois cada variável está perfeitamente correlacionada consigo mesma.

2. A correlação entre `bot_label` e `follower_count` é muito baixa (aproximadamente 0.001), indicando que não há uma relação linear significativa entre essas variáveis.

3. Correlação entre `follower_count` e `mention_count` é negativa (-0.007), sugerindo uma relação muito fraca e inversa entre essas variáveis.

4. Correlação entre `mention_count` e `retweet_count` também é negativa (-20.39952), o que parece ser um valor anômalo e pode indicar um erro nos dados ou na interpretação.


A presença de correlações negativas não significa que as variáveis não devam ser usadas. Uma correlação negativa indica que, à medida que uma variável aumenta, a outra tende a diminuir, por isso, nesse caso continuou-se com as variávies na construção do modelo.

# Modelo

Esse modelo está sendo construído para identificar se uma conta é um bot ou não. Primeiro, ele combina informações dos tweets, como o texto, localização e hashtags, em uma única variável para representar o conteúdo textual das postagens. Além disso, ele utiliza características adicionais, como ano de criação do tweet, número de retweets, menções e seguidores, para ajudar na classificação.


In [29]:
dados['text_combined'] = dados['tweet'] + ' ' + dados['location'] + ' ' + dados['hashtags']

# Separar as variáveis X e y
X_text = dados['text_combined'].values
X_features = dados[['created_at_year', 'retweet_count', 'mention_count', 'follower_count']].values
y = dados['bot_label'].values

# Dividir em treino e teste
X_text_train, X_text_test, X_feat_train, X_feat_test, y_train, y_test = train_test_split(
    X_text, X_features, y, test_size=0.3, random_state=42
)

Devido ao tamanho grande do dataset, foi utilizado apenas 10% dos dados para testar o modelo, o que permite uma avaliação inicial do desempenho sem comprometer a eficiência do treinamento. Caso seja necessário, é possível aumentar a quantidade de dados de teste para obter uma análise mais detalhada dos resultados.

In [30]:
# Utilizar apenas 10% dos dados
sample_size = int(0.1 * len(X_text_train))
X_text_train_subset = X_text_train[:sample_size]
y_train_subset = y_train[:sample_size]

Nesse modelo, está sendo utilizado o BERT (Bidirectional Encoder Representations from Transformers), que é uma técnica de processamento de linguagem natural desenvolvida pela Google. O BERT entende o contexto das palavras em um texto, pois analisa as palavras antes e depois de cada termo, o que melhora a compreensão do significado e a qualidade das previsões do modelo.

In [31]:
# Carregar o tokenizer do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenização para o subset do dataset
train_encodings_subset = tokenizer(list(X_text_train_subset), truncation=True, padding=True, max_length=64)
train_dataset_subset = BotDataset(train_encodings_subset, y_train_subset)

# Tokenizar o conjunto de teste completo
test_encodings = tokenizer(list(X_text_test), truncation=True, padding=True, max_length=64)
test_dataset = BotDataset(test_encodings, y_test)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



Esse trecho de código define um conjunto de dados personalizado (`BotDataset`) para ser usado no treinamento e avaliação do modelo. Ele armazena os textos tokenizados e os rótulos (`labels`), facilitando a forma como os dados são carregados. A função `__getitem__` retorna, para cada posição, as representações numéricas do texto e o rótulo correspondente. Já a função `__len__` informa o número total de exemplos no conjunto de dados.

In [32]:
# Criar um Dataset customizado para treinamento e avaliação
class BotDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

Esse trecho cria um conjunto de dados de treino (`train_dataset`) com os textos tokenizados e seus rótulos, e define o modelo de classificação de sequências usando o BERT.

In [33]:
# Criar datasets de treino e teste
train_dataset = BotDataset(train_encodings_subset, y_train_subset)

# Definir o modelo
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Abaixo são definidos os parâmetros de treinamento do modelo. O `TrainingArguments` especifica onde salvar os resultados, o número de épocas de treinamento, o tamanho do lote para treino e avaliação, além de etapas e taxa de decaimento de peso.

Em seguida, o `Trainer` é criado, que facilita o processo de treinamento e avaliação do modelo, utilizando os dados de treino e teste previamente definidos.

In [34]:

# Definir os argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    fp16=True,
)

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Treinar o modelo com o subset dos dados
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.671500,0.693381


TrainOutput(global_step=875, training_loss=0.7047227821350097, metrics={'train_runtime': 3374.6389, 'train_samples_per_second': 1.037, 'train_steps_per_second': 0.259, 'total_flos': 43166657520000.0, 'train_loss': 0.7047227821350097, 'epoch': 1.0})

A matriz de confusão é uma ferramenta que permite visualizar o desempenho de um modelo mostrando o número de acertos e erros nas previsões feitas.

In [41]:
# Matriz de Confusão
conf_matrix = confusion_matrix(y_test, y_pred)
labels = ['Not Bot', 'Bot']  # Alterar conforme o seu conjunto de dados
fig_conf = ff.create_annotated_heatmap(
    z=conf_matrix,
    x=labels,
    y=labels,
    colorscale='Blues',
    showscale=True
)
fig_conf.update_layout(title_text='Confusion Matrix', title_x=0.5)
fig_conf.show()

O modelo treinado é avaliado utilizando o conjunto de teste. Um relatório de classificação, fornece métricas como precisão, recall e F1-score, permitindo uma análise detalhada do desempenho do modelo.

In [35]:
# Avaliar o modelo
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(-1)

# Relatório de classificação
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.66      7461
           1       0.49      0.00      0.00      7539

    accuracy                           0.50     15000
   macro avg       0.49      0.50      0.33     15000
weighted avg       0.49      0.50      0.33     15000



# Análise do Modelo

O modelo criado apresenta que durante o treinamento, a perda final foi de 0.7047, e a validação indicou uma perda de 0.6934, sugerindo que o modelo aprendeu algo, mas não tem resultados tão satisfatórios.

Na análise do desempenho, a precisão e o recall para a classe 0 (não-bots) foram 50% de precisão e um recall de 100%. No entanto, a classe 1 (bots) não apresentou recall, indicando que o modelo não conseguiu identificar corretamente nenhum caso dessa categoria, resultando em uma F1-score de 0.00.

A matriz de confusão revela que, de um total de 15.000 amostras, o modelo acertou 7.521 casos como não-bots e apenas 19 como bots, enquanto 7.442 casos de não-bots foram classificados erroneamente como bots, demonstrando uma clara dificuldade em identificar a classe positiva.

Esses resultados sugerem que o modelo requer ajustes adicionais, para melhorar sua capacidade de detecção e, assim, proporcionar um desempenho mais equilibrado entre as classes.

# **EXTRA - Recomendações Futuras**

Para melhorar o desempenho do modelo considere as seguintes mudanças futuras:



> Observação: É importante garantir que haja um processamento adequado para o modelo, pois o grande número de linhas no dataset pode demandar tempo para testes e validações.



1. Em vez de usar apenas 10% dos dados para treinamento, tentar incluir uma maior quantidade, o que pode ajudar o modelo a aprender melhor as características das classes.

2. Como a classe de bots apresentou desempenho insatisfatório, considerar técnicas de balanceamento, como *oversampling* da classe minoritária (bots) ou *undersampling* da classe majoritária (não-bots), para que o modelo tenha uma quantidade semelhante de exemplos de ambas as classes.

3. Realizar uma busca de hiperparâmetros para otimizar a taxa de aprendizado, o tamanho do lote e o número de épocas. Como *grid search* ou *random search*.

4. O modelo foi treinado por apenas uma época, uma vez que quando tentado rodar com mais ficou mais de 4h rodando e não finalizava o treinamento. Aumentar o número de épocas pode permitir que o modelo converja melhor.


Implementar essas recomendações pode contribuir para um desempenho mais adequado.